# Testing extracting OSM data using Osmium

In [2]:
import os, sys, time, importlib

import geopandas as gpd
import pandas as pd
import networkx as nx
import numpy as np
sys.path.append("../../../GOSTnets")
import GOSTnets as gn

# pip install osmium
# import osmium, logging
# import shapely.wkb as wkblib

from shapely.geometry import LineString, Point
import osmnx as ox

In [3]:
# This is a Jupyter Notebook extension which reloads all of the modules whenever you run the code
# This is optional but good if you are modifying and testing source code
%load_ext autoreload
%autoreload 2

In [4]:
from GOSTnets.load_traffic2 import *

vavuniya is a city in northern Sri Lanka, and this will be a small area for testing

In [5]:
traffic_simplified_df = gn.load_traffic2.generate_traffic_metrics("./1233300-Asia-Colombo.csv", "./1233300-Asia-Colombo.csv", "./1233303-Asia-Colombo.csv")

2
./1233300-Asia-Colombo.csv
finished reading ./1233300-Asia-Colombo.csv into dataframe
FILE ./1233303-Asia-Colombo.csv
1233303-Asia-Colombo_df
finished merging 1233303-Asia-Colombo_df into combined dataframe
calculating min, max, and mean values.
finished calculating min, max, and mean values. Printing traffic_simplified head
    FROM_NODE     TO_NODE  min_speed  max_speed  mean_speed
0  1148494884  4177608798       31.0       38.0   36.866071
1  1148495298  4137314867       57.0       57.0   57.000000
2  1242700523  6537570627       60.0       60.0   60.000000
3  1242730766  3377418986       46.0       46.0   46.000000
4  1243299175  3805435746       40.0       40.0   40.000000


In [6]:
traffic_simplified_df

,FROM_NODE,TO_NODE,min_speed,max_speed,mean_speed
0,1148494884,4177608798,31.0,38.0,36.866071
1,1148495298,4137314867,57.0,57.0,57.000000
2,1242700523,6537570627,60.0,60.0,60.000000
3,1242730766,3377418986,46.0,46.0,46.000000
4,1243299175,3805435746,40.0,40.0,40.000000
...,...,...,...,...,...
34172,6495299720,6495344002,36.0,60.0,46.133929
34173,6495398243,6495398223,20.0,26.0,25.888393
34174,6899782766,3317976522,10.0,11.0,10.977679
34175,7082570748,7082570747,64.0,70.0,66.014881


In [7]:
# set file
some_file = './sri-lanka-latest.osm.pbf'

In [8]:
sri_lanka = OSM_to_network(some_file,traffic_simplified_df)

hit exception


hit exception
way 812734222 may not have nodes
finished with Osmium data extraction
234441
1
Error adding edge between nodes 5770924646 and 2907768386
{'osmid': 609259139, 'nodes': [2907768384, 5770924650, 5770924652, 6522171277, 2907768385, 7591229397, 7591229396, 7591229395, 5770924648, 5770924646, 2907768386], 'shp': <shapely.geometry.linestring.LineString object at 0x7f4504184cd0>, 'highway': 'residential', 'maxspeed': '30'}
try to list coords
[(81.8258068, 7.4257978), (81.8257961, 7.425779), (81.8255241, 7.425605), (81.8252243, 7.4254249), (81.8251366, 7.425373), (81.8250883, 7.4253424), (81.8250534, 7.4253218), (81.825034, 7.42531), (81.8250217, 7.4253038), (81.8250219, 7.4253038)]
length of nodes
11
print index
9
(81.8250219, 7.4253038)
checkpoint reached
Error adding edge between nodes 5770924651 and 2907768384
{'osmid': 609259140, 'nodes': [2907768383, 5770930354, 5770930356, 7591229329, 5770930353, 5770924651, 2907768384], 'shp': <shapely.geometry.linestring.LineString object

In [9]:
len(sri_lanka.network.edges())

8821281

In [10]:
# add length (great circle distance between nodes) attribute to each edge to
# use as weight
#if len(sri_lanka.network.edges) > 0:
G = ox.utils_graph.add_edge_lengths(sri_lanka.network)

In [11]:
len(G.nodes())

4350961

In [12]:
#fig, ax = ox.plot_graph(G, node_zorder=2, node_color='w', bgcolor='k')

### take the largest sub-graph

In [13]:
G = ox.utils_graph.get_largest_component(G)

In [14]:
len(G.edges())

8650707

In [16]:
# save graph for now

In [ ]:
gn.save(G,'sri_lanka_processed_graph_uncleaned_20200611','./', pickle = True, edges = True, nodes = True)

### removing all of the in-between nodes

In [ ]:
start = time.time()
G_clean = gn.clean_network(G, UTM = 'epsg:32644', WGS = 'epsg:4326', junctdist = 10, verbose = False)
end = time.time()
print(end - start)

finished with simplify_junctions
print node_list
17301414
print no_traffic_node_list
17233896


In [ ]:
fig, ax = ox.plot_graph(G_clean, node_zorder=2, node_color='w', bgcolor='k')

In [ ]:
len(G_clean.edges())

In [ ]:
len(G_clean)

### consolidate intersections

In [ ]:
G_proj = ox.project_graph(G_clean)

In [ ]:
G2 = ox.consolidate_intersections(G_proj, rebuild_graph=True, tolerance=15, dead_ends=False)
len(G2)

In [ ]:
fig, ax = ox.plot_graph(G2, node_zorder=2, node_color='w', bgcolor='k')

In [ ]:
gn.save(G2,'sri_lanka_processed_graph_before_speeds_20200610','./', pickle = True, edges = True, nodes = True)